<a href="https://colab.research.google.com/github/JonathanCheng123/AmazonLSTMNet/blob/main/Team_E_AmazonReview_LSTMNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TEAM E - AMAZON REVIEWS

In this project you work on Amazon food review from customers, and try to predict whether a review is positive or negative. 

**Dataset:**

The dataset contains more than 500K reviews with scores given by customers, number of upvotes & total votes to those comments. The dataset has ‘Score’ label which is given by customers to each product. Scores range from 1-5. 

For doing a binary classification, you need to get rid of score ‘3’ reviews (neutral) and separate the remaining reviews into binary class (1 = positive, 0 = negative)

**Size:** 394,375 instances

**Classes:** 2 (Positive = 1 , Negative = 0 )

**Class skewness:**

instance belonging to Negative class: 61,625

instances belonging to Positive class: 332,750

**Note: Reviews are skewed towards positive.**


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
%matplotlib inline 
import matplotlib.pyplot as plt
import gc
import string
import nltk
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
import keras
import gensim

from nltk.corpus import stopwords
nltk.download('stopwords')
stop = set(stopwords.words('english'))

from tensorflow.keras import layers
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
text = pd.read_csv("/content/drive/My Drive/Cosmos/train.csv",delimiter='\t', usecols=[10])
scored = pd.read_csv("/content/drive/My Drive/Cosmos/train.csv",delimiter='\t', usecols=[7])
test = pd.read_csv("/content/drive/My Drive/Cosmos/train.csv",delimiter='\t', usecols=[10])
testscored = pd.read_csv("/content/drive/My Drive/Cosmos/train.csv",delimiter='\t', usecols=[7])

In [ ]:
lemmatizer = WordNetLemmatizer()
def preprocess(s):
  return lemmatizer.lemmatize(s)

In [ ]:
cutoff=120000
scorecutoff=50000
features=15000
vocab_size = 10000
max_length = 16952 

In [ ]:
reviews=text.Text.tolist()
testreviews = test.Text.tolist()
score = scored.to_numpy().reshape(len(scored),1,1)
testscore = testscored.to_numpy().reshape(len(testscored),1,1)

In [ ]:
score1 = []
reviews1=[]
c=0
for i in range(len(reviews)):
  if(score[i]==0):
    if(c<30000):
      c += 1
      score1.append(score[i])
      reviews1.append(reviews[i])
c=0    
for i in range(len(reviews)):
  if(score[i]==1):
    if(c<90000):
      c += 1
      score1.append(score[i])
      reviews1.append(reviews[i])
score = score1
reviews = reviews1

In [ ]:
vectorizer = TfidfVectorizer(stop_words=stop, analyzer = 'word',max_features=features, dtype=np.float32, preprocessor=preprocess)
vectorizer.fit(text.Text)

score = np.array(score)
testscore = testscored.to_numpy()[0:scorecutoff].reshape(scorecutoff,1,1)

X_trainv = np.array([vectorizer.transform([a]).astype('float32').toarray() for a in reviews])
X_testv = np.array([vectorizer.transform([a]).astype('float32').toarray() for a in test.Text.tolist()[:scorecutoff]])

In [ ]:
X_train = np.squeeze(X_trainv)
X_train=X_train.reshape((len(X_train),1,features))
X_test = np.squeeze(X_testv).reshape(scorecutoff, 1, features)

In [ ]:
model = tf.keras.Sequential()
model.add(layers.LSTM(50, input_shape=(1,15000),return_sequences = True, recurrent_regularizer=tf.keras.regularizers.l2(0.005), kernel_initializer = 'glorot_normal', activation = 'relu'))
# model.add(layers.LSTM(64, return_sequences = True, kernel_initializer = 'glorot_normal', activation = 'relu', recurrent_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(layers.TimeDistributed(layers.Dense(16,activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.005))))
model.add(layers.TimeDistributed(layers.Dense(32,activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',metrics=['accuracy'],optimizer=tf.train.AdamOptimizer(0.0002))

In [ ]:
model.summary()
print ("Inputs: {}".format(model.input_shape))
print ("Outputs: {}".format(model.output_shape))
print ("Actual input: {}".format(X_train.shape))
print ("Actual output: {}".format(score.shape))

In [ ]:
a=model.fit(X_train, score, epochs=15,batch_size=256, validation_data=[X_test, testscore])

In [ ]:
model.evaluate(X_test,testscore,batch_size=32)

In [ ]:
val_acc = a.history['val_acc']
train_acc = a.history['acc']
epoch_count = range(1, len(val_acc) + 1)
plt.plot(epoch_count, train_acc, 'b')
plt.plot(epoch_count, val_acc, 'r')
plt.legend(['Training Accuracy', 'Test Accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show();

In [ ]:
training_loss = a.history['loss']
val_loss = a.history['val_loss']
epoch_count = range(1, len(val_loss) + 1)
plt.plot(epoch_count, val_loss, 'b')
plt.plot(epoch_count, training_loss, 'r')
plt.legend(['Validation Loss', 'Training Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show();

In [ ]:
def process(a):
  v=vectorizer.transform([a]).toarray()
  v=v.reshape(1,1,features)
  return v

string = input('Enter a review: ')
a = model.predict(process(string),batch_size=1,verbose = 0)
print(a)
if(a[0]>0.5):
  print("Positive review")
else:
  print("Negative review")

In [ ]:
model.save('drive/My Drive/model')

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('drive/My Drive/model')

In [ ]:
print(text[0])
print(textarray[0])